In [1]:
#importar librerias
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin
import pandas as pd
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
#cargar navegador y guardar pagina web en una variable
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
url = "http://books.toscrape.com/"
#listas vacias
titulo_sucio = []
precio_sucio = []
stock_sucio = []
titulos = []
precios = []
stocks = []

while True:
    #obtener código html a bs4
    driver.get(url)
    contenido = driver.page_source
    soup = bs(contenido)
    #crear for para extraer información y agregarla en las listas_sucio
    for libro in soup.find_all("article", attrs={"class":"product_pod"}):
        titulo = libro.find("h3")
        titulo_sucio.append(titulo.text)

        precio = libro.find("p", attrs={"class": "price_color"})
        precio_sucio.append(precio.text)

        stock = libro.find("p", attrs={"class": "instock availability"})
        stock_sucio.append(stock.text)
    #crear for para limpiar listas_sucio y agregar en listas
    for libro in titulo_sucio:
        titulos.append(libro.replace("\n","").strip())

    for libro in precio_sucio:
        precios.append(libro.replace("\n","").strip())

    for libro in stock_sucio:
        stocks.append(libro.replace("\n","").strip())

    #saltar a la siguiente pagina y recolectar datos
    siguiente_pag = soup.select_one("li.next>a")
    if siguiente_pag:
        siguiente_url = siguiente_pag.get("href")
        url = urljoin(url, siguiente_url)
    else:
        break

#guardar datos en un DataFrame y exportar a csv
df = pd.DataFrame({"Titulo": titulos, "Precio": precios, "Stock": stocks})
df.to_csv("Data_extraida.csv", index=False, encoding="utf-8")